In [2]:
"""
A very simple and basic web scraping script. Feel free to
use this as a source of inspiration, but, make sure to attribute
it if you do so.

This is by no means production code.
"""
# built-in imports
import re
from json import dump
import requests
headers = {"User-Agent": 
           "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
from collections import defaultdict

# user packages
from bs4 import BeautifulSoup
from urllib.request import urlopen

# constants
BASE_URL = "https://www.domain.com.au/"
N_PAGES = range(1, 51) # update this to your liking
PRICE_BIN = ["0-300", "300-350", "350-400", "400-450", "450-500", "500-550", "550-600", "600-650", "650-800", "800-any"]

# begin code
url_links = []
property_metadata = defaultdict(dict)

for price in PRICE_BIN:
# generate list of urls to visit
    for page in N_PAGES:
        url = BASE_URL + f"rent/?price={price}&sort=price-desc&state=vic&page={page}"
        bs_object = BeautifulSoup(requests.get(url, headers=headers).text, "html.parser")

        # find the unordered list (ul) elements which are the results, then
        # find all href (a) tags that are from the base_url website.
        index_links = bs_object \
            .find(
                "ul",
                {"data-testid": "results"}
            ) \
            .findAll(
                "a",
                href=re.compile(f"{BASE_URL}/*") # the `*` denotes wildcard any
            )
        for link in index_links:
            # if its a property address, add it to the list

            if 'address' in link['class']:
                url_links.append(link['href'])
i = 0
tot = len(url_links)
# for each url, scrape some basic metadata
for property_url in url_links[1:]:
    i=i+1
    if(i%10 == 0):
        print(f"progress: {i} of {tot}")
    bs_object = bs_object = BeautifulSoup(requests.get(property_url, headers=headers).text, "html.parser")
    print(property_url)
    
    # looks for the header class to get property name
    name = bs_object \
        .find("h1", {"class": "css-164r41r"}) 
    if(name):
        property_metadata[property_url]['name'] = name.text
    else:
        property_metadata[property_url]['name'] = ""
        
        
    # looks for the div containing a summary title for cost
    cost = bs_object \
        .find("div", {"data-testid": "listing-details__summary-title"}) 
    if(cost):
        property_metadata[property_url]['cost_text'] = cost.text
    else:
        property_metadata[property_url]['cost_text'] = ""

    property_type = bs_object \
        .find("div", {"data-testid": "listing-summary-property-type"}) 
    if(property_type):
        type_span = property_type \
            .find("span", {"class": "css-in3yi3"})
        if(type_span):
            property_metadata[property_url]['type'] = type_span.text
        else:
            property_metadata[property_url]['type'] = ""
    else: 
        property_metadata[property_url]['type'] = ""

    
    ulTag = bs_object \
        .find("div", {"data-testid": "strip-content-list"})
    area = 0
    if(ulTag):
        ulTag = ulTag.find("ul" , {"data-testid": "listing-summary-strip"})
        for litag in ulTag:
            if "area" in litag.text:
                curSize = re.findall(r'[0-9]+', litag.text)
                area += int(curSize[0])
    property_metadata[property_url]['Area'] = area

    # extract coordinates from the hyperlink provided
    # i'll let you figure out what this does :P
    try:
        property_metadata[property_url]['coordinates'] = [
            float(coord) for coord in re.findall(
                r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
                bs_object \
                    .find(
                        "a",
                        {"target": "_blank", 'rel': "noopener noreferer"}
                    ) \
                    .attrs['href']
            )[0].split(',')
        ]
    except AttributeError:
        property_metadata[property_url]['coordinates'] = [0,0]

    rooms = []
    for feature in bs_object \
            .find("div", {"data-testid": "property-features"}) \
            .findAll("span", {"data-testid": "property-features-text-container"}):
        if(re.findall(r'\d\s[A-Za-z]+', feature.text)):
            rooms.append(re.findall(r'\d\s[A-Za-z]+', feature.text)[0])
            
        property_metadata[property_url]['rooms'] = rooms
        
# output to example json in data/raw/
with open('../data/raw/property.json', 'w') as f:
    dump(property_metadata, f)

https://www.domain.com.au/56-selwyn-street-albion-vic-3020-16051103
https://www.domain.com.au/1-3-robin-street-altona-vic-3018-16066708
https://www.domain.com.au/34-hucker-street-ararat-vic-3377-16067682
https://www.domain.com.au/3-neylan-street-ararat-vic-3377-16067634
https://www.domain.com.au/8-suspension-street-ardeer-vic-3022-16041068
https://www.domain.com.au/14-27-st-georges-road-armadale-vic-3143-14856011
https://www.domain.com.au/1-28-ormond-road-ascot-vale-vic-3032-16052047
https://www.domain.com.au/30b-san-remo-drive-avondale-heights-vic-3034-15973149
https://www.domain.com.au/205-johns-street-ballarat-east-vic-3350-15991244
progress: 10 of 10000
https://www.domain.com.au/14b-purcell-st-benalla-vic-3672-15452963
https://www.domain.com.au/3-25a-havlin-street-bendigo-vic-3550-16046289
https://www.domain.com.au/3-180-williamson-street-bendigo-vic-3550-16061472
https://www.domain.com.au/765-warrigal-rd-bentleigh-east-vic-3165-15462384
https://www.domain.com.au/4-718-chisholm-str

https://www.domain.com.au/71-hourigan-road-morwell-vic-3840-15988478
https://www.domain.com.au/2-130-bridle-road-morwell-vic-3840-16046249
https://www.domain.com.au/2-4-rathkeale-avenue-mount-helen-vic-3350-16042951
https://www.domain.com.au/5a-44-60-jacksons-road-mulgrave-vic-3170-15983879
https://www.domain.com.au/3-24-brisbane-street-murrumbeena-vic-3163-16060227
https://www.domain.com.au/2-15-gordon-street-noble-park-vic-3174-16017376
https://www.domain.com.au/1-1-5-lawn-road-noble-park-vic-3174-15053268
https://www.domain.com.au/2-14-alamein-street-noble-park-vic-3174-11845762
https://www.domain.com.au/2-5-duncan-court-noble-park-vic-3174-16053915
https://www.domain.com.au/11-ulladulla-street-norlane-vic-3214-16049973
progress: 120 of 10000
https://www.domain.com.au/50-wendover-avenue-norlane-vic-3214-16054891
https://www.domain.com.au/2-26-canning-street-north-melbourne-vic-3051-16063485
https://www.domain.com.au/202-179-boundary-road-north-melbourne-vic-3051-16070062
https://www

https://www.domain.com.au/2-217-buckley-street-seddon-vic-3011-15850994
progress: 220 of 10000
https://www.domain.com.au/18-oak-street-shepparton-vic-3630-10846955
https://www.domain.com.au/313-kings-way-south-melbourne-vic-3205-15691244
https://www.domain.com.au/1-mcarthur-avenue-st-albans-vic-3021-16048024
https://www.domain.com.au/9-40-waterloo-crescent-st-kilda-vic-3182-15772227
https://www.domain.com.au/9-40-waterloo-crescent-st-kilda-vic-3182-15934567
https://www.domain.com.au/6-30-westbury-street-st-kilda-east-vic-3183-15132614
https://www.domain.com.au/7-130-alexandra-street-st-kilda-east-vic-3183-16063528
https://www.domain.com.au/3-hassett-street-sunshine-north-vic-3020-16069347
https://www.domain.com.au/4-145-smith-st-thornbury-vic-3071-13781409
https://www.domain.com.au/2614-murray-valley-highway-torrumbarry-vic-3562-15860318
progress: 230 of 10000
https://www.domain.com.au/64-molyneaux-street-warracknabeal-vic-3393-16021707
https://www.domain.com.au/13-austin-street-werrib

https://www.domain.com.au/24-3-holmes-street-brunswick-east-vic-3057-14697446
https://www.domain.com.au/a42-1-5-grantham-street-brunswick-west-vic-3055-16059411
https://www.domain.com.au/2-99-neerim-road-carnegie-vic-3163-11936047
progress: 330 of 10000
https://www.domain.com.au/5-19-riches-street-dallas-vic-3047-16070727
https://www.domain.com.au/1-1-william-avenue-dandenong-south-vic-3175-15559520
https://www.domain.com.au/10-156a-napier-st-essendon-vic-3040-15990313
https://www.domain.com.au/4-6-koormiel-lane-fitzroy-north-vic-3068-16058937
https://www.domain.com.au/footscray-vic-3011-16059455
https://www.domain.com.au/2-17-lewis-street-frankston-vic-3199-15462611
https://www.domain.com.au/2-99-neerim-road-glen-huntly-vic-3163-11935961
https://www.domain.com.au/2-sycamore-street-hamlyn-heights-vic-3215-15052622
https://www.domain.com.au/4-1-king-street-hastings-vic-3915-16070175
https://www.domain.com.au/11-1-warida-avenue-malvern-east-vic-3145-16025080
progress: 340 of 10000
https:

https://www.domain.com.au/89a-revell-street-blairgowrie-vic-3942-15297789
https://www.domain.com.au/787-park-st-brunswick-vic-3056-7181433
https://www.domain.com.au/25-3-holmes-street-brunswick-east-vic-3057-12620149
https://www.domain.com.au/30-3-holmes-street-brunswick-east-vic-3057-12639737
https://www.domain.com.au/11-3-holmes-street-brunswick-east-vic-3057-12613234
progress: 440 of 10000
https://www.domain.com.au/27-3-holmes-street-brunswick-east-vic-3057-12685230
https://www.domain.com.au/14-3-holmes-street-brunswick-east-vic-3057-12685197
https://www.domain.com.au/4-24-daly-street-brunswick-west-vic-3055-15572002
https://www.domain.com.au/11-13-church-street-campbellfield-vic-3061-15956312
https://www.domain.com.au/2-36-woornack-road-carnegie-vic-3163-16054630
https://www.domain.com.au/3-54-narong-road-caulfield-north-vic-3161-16029918
https://www.domain.com.au/2-61-eskdale-road-caulfield-north-vic-3161-16046268
https://www.domain.com.au/28-mcleod-street-coleraine-vic-3315-16025

...

https://www.domain.com.au/45-cityview-road-balwyn-north-vic-3104-15947092
https://www.domain.com.au/264-centre-road-bentleigh-vic-3204-15825305
progress: 9530 of 10000
https://www.domain.com.au/12b-mckittrick-street-bentleigh-vic-3204-15977980
https://www.domain.com.au/26-eliza-street-black-rock-vic-3193-16052453
https://www.domain.com.au/112-south-rd-brighton-east-vic-3187-14865595
https://www.domain.com.au/78-marriage-rd-brighton-east-vic-3187-16065745
https://www.domain.com.au/16-valdemar-court-brighton-east-vic-3187-15999262
https://www.domain.com.au/13-35-stewart-street-brunswick-vic-3056-16052842
https://www.domain.com.au/222-wattle-valley-rd-camberwell-vic-3124-16069348
https://www.domain.com.au/4-22-grattan-street-carlton-vic-3053-15844687
https://www.domain.com.au/16-david-street-carlton-vic-3053-15965655
https://www.domain.com.au/carlton-vic-3053-16069887
progress: 9540 of 10000
https://www.domain.com.au/18-drummond-street-carlton-vic-3053-14908450
https://www.domain.com.au/2

https://www.domain.com.au/4-palfreyman-street-south-yarra-vic-3141-16021762
https://www.domain.com.au/619-1-freshwater-place-southbank-vic-3006-15501871
https://www.domain.com.au/3-22b-south-road-brighton-vic-3186-16065921
progress: 9640 of 10000
https://www.domain.com.au/42-o-grady-street-clifton-hill-vic-3068-16059929
https://www.domain.com.au/19-henry-street-kensington-vic-3031-16050886
https://www.domain.com.au/73-holtom-street-west-princes-hill-vic-3054-15008416
https://www.domain.com.au/27b-park-street-seaford-vic-3198-15933374
https://www.domain.com.au/1-2-howell-street-brighton-east-vic-3187-16070077
https://www.domain.com.au/16b-keswick-street-bentleigh-east-vic-3165-16064810
https://www.domain.com.au/231-barkly-street-brunswick-vic-3056-16021666
https://www.domain.com.au/107-field-street-clifton-hill-vic-3068-14758704
https://www.domain.com.au/2-14-sylvan-street-balwyn-north-vic-3104-15721267
https://www.domain.com.au/82-downey-road-dewhurst-vic-3808-16051941
progress: 9650 o

https://www.domain.com.au/3745-point-nepean-road-portsea-vic-3944-15895819
https://www.domain.com.au/48-yarra-street-abbotsford-vic-3067-16069226
https://www.domain.com.au/4-west-street-ascot-vale-vic-3032-16004955
https://www.domain.com.au/37-yerrin-street-balwyn-vic-3103-16015969
progress: 9750 of 10000
https://www.domain.com.au/8-mooroopna-court-barwon-heads-vic-3227-14855007
https://www.domain.com.au/11-13-tibbles-street-beaumaris-vic-3193-16054347
https://www.domain.com.au/11a-smith-st-bentleigh-vic-3204-12680078
https://www.domain.com.au/7b-bevis-street-bentleigh-east-vic-3165-15900569
https://www.domain.com.au/12-forest-glen-avenue-blackburn-south-vic-3130-16068557
https://www.domain.com.au/2501-828-whitehorse-road-box-hill-vic-3128-15999360
https://www.domain.com.au/82-laura-street-brunswick-vic-3056-16071918
https://www.domain.com.au/34-david-street-brunswick-vic-3056-15784703
https://www.domain.com.au/42-albion-street-brunswick-east-vic-3057-16071972
https://www.domain.com.au

https://www.domain.com.au/670-rathdowne-street-carlton-north-vic-3054-15997081
https://www.domain.com.au/690-canning-street-carlton-north-vic-3054-16063345
https://www.domain.com.au/390-bay-road-cheltenham-vic-3192-15941471
https://www.domain.com.au/2-wedd-street-cheltenham-vic-3192-16009427
https://www.domain.com.au/1107-8-marmion-place-docklands-vic-3008-15258330
https://www.domain.com.au/2004-231-harbour-esp-docklands-vic-3008-15143045
progress: 9860 of 10000
https://www.domain.com.au/263-glenlyon-road-fitzroy-north-vic-3068-15946830
https://www.domain.com.au/2-burramine-road-glen-waverley-vic-3150-16065822
https://www.domain.com.au/167-auburn-road-hawthorn-vic-3122-16045989
https://www.domain.com.au/70-emo-road-malvern-east-vic-3145-16024074
https://www.domain.com.au/3815-228-la-trobe-street-melbourne-vic-3000-14985539
https://www.domain.com.au/13-balcombe-street-mornington-vic-3931-16055425
https://www.domain.com.au/14-john-william-drive-mount-martha-vic-3934-16066038
https://www.

https://www.domain.com.au/149-ludstone-street-hampton-vic-3188-16027982
https://www.domain.com.au/5-scotch-circuit-hawthorn-vic-3122-16042104
https://www.domain.com.au/474-burwood-road-hawthorn-vic-3122-16064070
https://www.domain.com.au/2a-garden-street-hawthorn-vic-3122-16056125
https://www.domain.com.au/1-26-redfern-road-hawthorn-east-vic-3123-16068361
https://www.domain.com.au/50-kyeema-avenue-highton-vic-3216-16022129
https://www.domain.com.au/5-80-hobsons-road-kensington-vic-3031-16050912
https://www.domain.com.au/147-kilby-road-kew-east-vic-3102-12058057
progress: 9970 of 10000
https://www.domain.com.au/58-wright-street-mckinnon-vic-3204-16041390
https://www.domain.com.au/709-14-queens-road-melbourne-vic-3004-16031590
https://www.domain.com.au/63-strabane-ave-mont-albert-north-vic-3129-16046501
https://www.domain.com.au/723-mt-alexander-road-moonee-ponds-vic-3039-16066798
https://www.domain.com.au/110-park-street-moonee-ponds-vic-3039-16009078
https://www.domain.com.au/29-dactyl